In [1]:
# Dependencies and Setup
import pandas as pd

# Study data files
GCSE_char_data_path = "Data_original/2223_national_char_data_provisional.csv"
GCSE_regional_data_path = "Data_original/2223_la_data_provisional.csv"
GCSE_national_data_path = "Data_original/2223_national_data_provisional.csv"
GCSE_subject_data_path = "Data_original/2223_subject_entry_level_data_provisional.csv"
GCSE_subject_timeseries_data_path = "Data_original/2223_subject_timeseries_data_provisional.csv"
employment_data_path = "Data_original/TS065_2021_Employment.csv"
qualifications_data_path = "Data_original/TS067_2021_Qualifications.csv"

# Read the mouse data and the study results
GCSE_char_data = pd.read_csv(GCSE_char_data_path)
GCSE_regional_data = pd.read_csv(GCSE_regional_data_path)
GCSE_national_data = pd.read_csv(GCSE_national_data_path)
GCSE_subject_data = pd.read_csv(GCSE_subject_data_path)
GCSE_subject_timeseries_data = pd.read_csv(GCSE_subject_timeseries_data_path)
employment_data = pd.read_csv(employment_data_path)
qualifications_data  = pd.read_csv(qualifications_data_path)

GCSE_subject_timeseries_data.head()

,time_period,time_identifier,geographic_level,country_code,country_name,version,establishment_type,gender,subject,total_exam_entries,grade,percentage_achieving
0,202223,Academic year,National,E92000001,England,Provisional,All schools,Boys,All Subjects,2381705,91AstarG,96.8
1,202223,Academic year,National,E92000001,England,Provisional,All schools,Girls,All Subjects,2354678,91AstarG,97.8
2,202223,Academic year,National,E92000001,England,Provisional,All schools,Total,All Subjects,4736383,91AstarG,97.3
3,202223,Academic year,National,E92000001,England,Provisional,All schools,Boys,All Subjects,2381705,94AstarC,67.5
4,202223,Academic year,National,E92000001,England,Provisional,All schools,Girls,All Subjects,2354678,94AstarC,74.7


In [3]:
# clean and sort the overall data which contains characteristics such as ethnicity, gender etc. 
# for the first analysis on ethnicity and social class

ethnicity_data_df = GCSE_char_data[(GCSE_char_data["establishment_type"] == "All state-funded") & ((GCSE_char_data["breakdown"] == "Ethnic major") | (GCSE_char_data["breakdown"] == "Free school meals") | (GCSE_char_data["breakdown"] == "Total")) \
                                   & (GCSE_char_data["sen_status"] == "Total") \
                                   & (GCSE_char_data["all_primary_need"] == "Total") \
                                   & (GCSE_char_data["sen_description"] == "Total") \
                                   & (GCSE_char_data["disadvantage"] == "Total") \
                                   & (GCSE_char_data["first_language"] == "Total") \
                                   & (GCSE_char_data["admission_type"] == "Total") \
                                   & (GCSE_char_data["religious_denomination"] == "Total")]  

ethnicity_data_reduced_df = ethnicity_data_df[["time_period","gender","ethnicity_major","ethnicity_minor","free_school_meals",
                                               "t_schools","t_pupils","avg_att8","avg_p8score","pt_entbasics",
                                               "pt_l2basics_95"]]

ethnicity_data_reduced_df = ethnicity_data_reduced_df.dropna(how="any")

ethnicity_data_reduced_df = ethnicity_data_reduced_df.rename(columns={"time_period":"Year",
                                                                      "t_schools": "total_num_schools",
                                                                      "t_pupils": "total_num_pupils",
                                                                      "avg_att8":"avg_attainment8_score",
                                                                      "avg_p8score":"avg_progress8_score",
                                                                      "pt_entbasics":"perc_entered_eng_maths",
                                                                      "pt_l2basics_95":"perc_achieving_grade_5to9_eng_maths"})

ethnicity_data_reduced_df = ethnicity_data_reduced_df.replace([201819,201920,202021,202122,202223],[2019,2020,2021,2022,2023])


ethnicity_data_reduced_df = ethnicity_data_reduced_df.replace(to_replace=":",value="0.00")

ethnicity_data_reduced_df["Year"] = ethnicity_data_reduced_df["Year"].astype(str)
ethnicity_data_reduced_df["avg_attainment8_score"] = ethnicity_data_reduced_df["avg_attainment8_score"].astype(float)
ethnicity_data_reduced_df["perc_entered_eng_maths"] = ethnicity_data_reduced_df["perc_entered_eng_maths"].astype(float)
ethnicity_data_reduced_df["perc_achieving_grade_5to9_eng_maths"] = ethnicity_data_reduced_df["perc_achieving_grade_5to9_eng_maths"].astype(float)
#ethnicity_data_reduced_df["avg_progress8_score"] = ethnicity_data_reduced_df["avg_progress8_score"].astype(float)

ethnicity_data_reduced_df.set_index("Year",inplace=True,drop="True")

#ethnicity_data_reduced_df[(ethnicity_data_reduced_df["Year"]=="2020")]

In [4]:
ethnicity_data_reduced_df.to_csv("Data_processed/cleaned_ethnicity_data.csv")

In [5]:
# clean and sort the overall data which contains characteristics such as ethnicity, gender etc. 
# for the second analysis on subjects entered at GCSE by gender

subject_data_df = GCSE_national_data[(GCSE_national_data["time_period"]==202223) & (GCSE_national_data["establishment_type"] == "All schools") & (GCSE_national_data["breakdown"] == "Total")]


subject_data_reduced_df = subject_data_df[["gender","t_schools","t_pupils",
                                           "t_entbasics","pt_entbasics","t_triplesci_e","pt_triplesci_e",
                                           "t_eballsci_ptq_ee","pt_eballsci_ptq_ee",
                                           "t_ent_comb_sci","pt_ent_comb_sci","t_ent_hist_geog","pt_ent_hist_geog",
                                           "t_entered_art","pt_entered_art","t_entered_music_qual","pt_entered_music_qual",
                                           "t_multilan_e","pt_multilan_e","avg_att8","avg_p8score"]]

subject_data_reduced_df = subject_data_reduced_df.dropna(how="any")

subject_data_reduced_df = subject_data_reduced_df.rename(columns={"t_schools": "total_num_schools", 
                                                                  "t_pupils": "total_num_pupils",
                                                                  "t_entbasics": "tot_num_entering_english_and_maths_GCSEs",
                                                                  "pt_entbasics":"perc_pupils_entering_english_and_maths_GCSEs",
                                                                  "t_triplesci_e":"tot_num_entering_triple_science",
                                                                  "pt_triplesci_e":"perc_pupils_entering_triple_science",
                                                                  "t_eballsci_ptq_ee":"tot_num_entering_one_of_phys_chem_bio_compsci",
                                                                  "pt_eballsci_ptq_ee":"perc_pupils_entering_one_of_phys_chem_bio_compsci",
                                                                  "t_ent_comb_sci":"tot_num_entering_combined_science",
                                                                  "pt_ent_comb_sci":"perc_pupils_entering_combined_science",
                                                                  "t_ent_hist_geog":"tot_num_entering_hist_and_geog",
                                                                  "pt_ent_hist_geog":"perc_pupils_entering_hist_and_geog",
                                                                  "t_entered_art":"tot_num_entering_any_arts_subj",
                                                                  "pt_entered_art":"perc_pupils_entering_any_arts_subj",
                                                                  "t_entered_music_qual":"tot_num_entering_music_qualification",
                                                                  "pt_entered_music_qual":"perc_pupils_entering_music_qualification",
                                                                  "t_multilan_e":"tot_num_entering_more_than_one_lang",
                                                                  "pt_multilan_e":"perc_pupils_entering_more_than_one_lang",
                                                                  "avg_att8":"avg_attainment8_score:",
                                                                  "avg_p8score":"avg_progress8_score:"})

subject_data_reduced_df = subject_data_reduced_df.reset_index(drop=True)
subject_data_reduced_df.head()


,gender,total_num_schools,total_num_pupils,tot_num_entering_english_and_maths_GCSEs,perc_pupils_entering_english_and_maths_GCSEs,tot_num_entering_triple_science,perc_pupils_entering_triple_science,tot_num_entering_one_of_phys_chem_bio_compsci,perc_pupils_entering_one_of_phys_chem_bio_compsci,tot_num_entering_combined_science,...,tot_num_entering_hist_and_geog,perc_pupils_entering_hist_and_geog,tot_num_entering_any_arts_subj,perc_pupils_entering_any_arts_subj,tot_num_entering_music_qualification,perc_pupils_entering_music_qualification,tot_num_entering_more_than_one_lang,perc_pupils_entering_more_than_one_lang,avg_attainment8_score:,avg_progress8_score:
0,Boys,5258,345201,306059,88.7,85025,24.6,85452,24.8,219593,...,32556,9.4,97520,28.3,24138,7,10205,3,42.1,-0.21
1,Girls,5213,326255,296793,91,81828,25.1,81943,25.1,215215,...,20294,6.2,160144,49.1,21866,6.7,13723,4.2,47.2,0.1
2,Total,5717,671456,602852,89.8,166853,24.8,167395,24.9,434808,...,52850,7.9,257664,38.4,46004,6.9,23928,3.6,44.6,-0.06


In [6]:
subject_data_reduced_df.to_csv("Data_processed/cleaned_subject_data.csv")

In [7]:
#all_subject_data_df = GCSE_subject_timeseries_data[(GCSE_subject_timeseries_data["time_period"]>201718)]
all_subject_data_df = GCSE_subject_timeseries_data[(GCSE_subject_timeseries_data["time_period"]>201718)]
all_subject_data_reduced_df = all_subject_data_df[["time_period","subject","gender","total_exam_entries","grade","percentage_achieving"]]

all_subject_data_reduced_df


,time_period,subject,gender,total_exam_entries,grade,percentage_achieving
0,202223,All Subjects,Boys,2381705,91AstarG,96.8
1,202223,All Subjects,Girls,2354678,91AstarG,97.8
2,202223,All Subjects,Total,4736383,91AstarG,97.3
3,202223,All Subjects,Boys,2381705,94AstarC,67.5
4,202223,All Subjects,Girls,2354678,94AstarC,74.7
...,...,...,...,...,...,...
10060,201819,Urdu,Girls,1905,G,z
10061,201819,Urdu,Total,3038,G,z
10062,201819,Urdu,Boys,1133,Covid impacted,z
10063,201819,Urdu,Girls,1905,Covid impacted,z


In [8]:
# clean and sort the national data for the third analysis on GCSE outcomes by type of school attended

school_data_df = GCSE_national_data[(GCSE_national_data["time_period"]==202223) & (GCSE_national_data["breakdown"] == "Total")]

school_data_reduced_df = school_data_df[["establishment_type","gender","t_schools","t_pupils","avg_att8","avg_p8score","pt_l2basics_95"]]

school_data_reduced_df = school_data_reduced_df.dropna(how="any")
school_data_reduced_df = school_data_reduced_df.rename(columns={"t_schools": "total_num_schools",
                                                                      "t_pupils": "total_num_pupils",
                                                                      "avg_att8":"avg_attainment8_score",
                                                                      "avg_p8score":"avg_progress8_score",
                                                                      "pt_l2basics_95":"perc_achieving_grade_5to9_eng_maths"})

school_data_reduced_df = school_data_reduced_df.reset_index(drop=True)

school_data_reduced_df.head()


,establishment_type,gender,total_num_schools,total_num_pupils,avg_attainment8_score,avg_progress8_score,perc_achieving_grade_5to9_eng_maths
0,Academies and free schools,Boys,2450,238905,45.3,-0.12,44.4
1,Academies and free schools,Girls,2481,231994,49.2,0.14,48
2,Academies and free schools,Total,2601,470899,47.2,0.01,46.2
3,All independent schools,Boys,1032,27596,30.7,-1.82,15
4,All independent schools,Girls,1024,25462,38.2,-1.56,20.2


In [9]:
school_data_reduced_df.to_csv("Data_processed/cleaned_school_data.csv")

In [10]:
# clean and sort the local data for the fourth analysis on GCSE outcomes by area

regional_data_df = GCSE_regional_data[(GCSE_regional_data["time_period"]==202223)]

regional_data_reduced_df = regional_data_df[["la_name","gender","t_schools","t_pupils","avg_att8","avg_p8score","pt_l2basics_95"]]

regional_data_reduced_df = regional_data_reduced_df.dropna(how="any")

regional_data_reduced_df = regional_data_reduced_df.rename(columns={"t_schools": "total_num_schools",
                                                                      "t_pupils": "total_num_pupils",
                                                                      "avg_att8":"avg_attainment8_score",
                                                                      "avg_p8score":"avg_progress8_score",
                                                                      "pt_l2basics_95":"perc_achieving_grade_5to9_eng_maths"})

regional_data_reduced_df = regional_data_reduced_df.reset_index(drop=True)

regional_data_reduced_df.head()

,la_name,gender,total_num_schools,total_num_pupils,avg_attainment8_score,avg_progress8_score,perc_achieving_grade_5to9_eng_maths
0,Hartlepool,Boys,6,568,38.5,-0.64,34.3
1,Hartlepool,Girls,6,523,44.2,-0.27,41.1
2,Hartlepool,Total,6,1091,41.2,-0.46,37.6
3,Middlesbrough,Boys,10,791,37.2,-0.64,31.2
4,Middlesbrough,Girls,9,809,42.8,-0.29,36.6


In [11]:
regional_data_reduced_df.to_csv("Data_processed/cleaned_regional_data.csv")

In [13]:
# clean and sort the local data for the fifth analysis on GCSE outcomes before and after COVID

time_data_df = GCSE_national_data[(GCSE_national_data["time_period"]>=201819) & ((GCSE_national_data["establishment_type"] == "All schools") | (GCSE_national_data["establishment_type"] == "All state-funded") | (GCSE_national_data["establishment_type"] == "All independent schools")) & (GCSE_national_data["breakdown"] == "Total") &   (GCSE_national_data["gender"] == "Total")]

time_data_reduced_df = time_data_df[["time_period","establishment_type","t_schools","t_pupils","avg_att8"]]

time_data_reduced_df = time_data_reduced_df.dropna(how="any")

time_data_reduced_df = time_data_reduced_df.rename(columns={"t_schools": "total_num_schools",
                                                                      "t_pupils": "total_num_pupils",
                                                                      "t_schools": "total_num_schools",
                                                                      "avg_att8":"avg_attainment8_score",
                                                                      "avg_p8score":"avg_progress8_score"})
time_data_reduced_df = time_data_reduced_df.reset_index(drop=True)

time_data_reduced_df

,time_period,establishment_type,total_num_schools,total_num_pupils,avg_attainment8_score
0,202223,All independent schools,1253,53058,34.3
1,202223,All schools,5717,671456,44.6
2,202223,All state-funded,4047,606947,46.2
3,202122,All independent schools,1226,50678,34.6
4,202122,All schools,5667,648647,47
5,202122,All state-funded,4015,587660,48.8
6,202021,All independent schools,1204,49597,36.8
7,202021,All schools,5626,638824,48.9
8,202021,All state-funded,3989,575863,50.9
9,201920,All state-funded,3987,561994,50.2


In [14]:
time_data_reduced_df.to_csv("Data_processed/cleaned_time_data.csv")

In [15]:
# clean and sort the local data for the sixth analysis, how do grade 9s achieved differ across the subjects

grade9_data_df = GCSE_subject_data[(GCSE_subject_data["grade"]=="9") | (GCSE_subject_data["grade"]=="Total exam entries")]

mask = grade9_data_df['number_achieving'] == 'z'
grade9_data_df = grade9_data_df[~mask]

grade9_data_reduced_df =grade9_data_df[["subject","grade","number_achieving"]]

grade9_data_reduced_df = grade9_data_reduced_df.reset_index(drop=True)

grade9_data_reduced_df

,subject,grade,number_achieving
0,All Subjects,Total exam entries,4736478
1,Ancient History,Total exam entries,1219
2,Ancient History,9,148
3,Any classical study,Total exam entries,14133
4,Any classical study,9,4054
...,...,...,...
76,Social Studies,9,2383
77,Spanish,Total exam entries,120059
78,Spanish,9,7486
79,Statistics,Total exam entries,25637


In [16]:
grade9_data_reduced_df.to_csv("Data_processed/cleaned_grade9_data.csv")

In [23]:
# Extract additional data for subject analysis

subj_data_all_years_df =GCSE_subject_timeseries_data[(GCSE_subject_timeseries_data["grade"]=="9 to 5")]

subj_data_all_years_red_df = subj_data_all_years_df[["time_period","gender","subject","total_exam_entries","grade"]]

subj_data_all_years_red_df


,time_period,gender,subject,total_exam_entries,grade
9,202223,Boys,All Subjects,2381705,9 to 5
10,202223,Girls,All Subjects,2354678,9 to 5
11,202223,Total,All Subjects,4736383,9 to 5
126,202223,Boys,Ancient History,675,9 to 5
127,202223,Girls,Ancient History,544,9 to 5
...,...,...,...,...,...
35161,200910,Girls,Urdu,2791,9 to 5
35162,200910,Total,Urdu,4550,9 to 5
35199,200910,Boys,Vocational Studies,20355,9 to 5
35200,200910,Girls,Vocational Studies,21321,9 to 5


In [24]:
subj_data_all_years_red_df.to_csv("Data_processed/cleaned_subject_data_all_years.csv")

In [3]:
# Extract employment and educational qualifications for areas around the UK

qualifications_data_df = qualifications_data[(qualifications_data["Highest level of qualification (8 categories)"]=="No qualifications")]
qualifications_data_df = qualifications_data_df[["Lower tier local authorities","Highest level of qualification (8 categories)","Observation"]]
qualifications_data_df = qualifications_data_df.rename(columns={"Lower tier local authorities":"Local authority",
                                                                "Highest level of qualification (8 categories)":"Highest level of qualification",
                                                                "Observation":"Number of people"})

qualifications_data_df.to_csv("Data_processed/qualifications_data_for_local_authorities.csv")

employment_data_df = employment_data[(employment_data["Employment history (4 categories)"]=="Not in employment: Never worked")]
employment_data_df = employment_data_df[["Lower tier local authorities","Employment history (4 categories)","Observation"]]
employment_data_df = employment_data_df.rename(columns={"Lower tier local authorities":"Local authority",
                                                                "Employment history (4 categories)":"Employment history",
                                                                "Observation":"Number of people"})

employment_data_df.to_csv("Data_processed/employment_data_for_local_authorities.csv")